### Importing Libraries

In [22]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 2.6MB/s 


### Scrap Data from Wikipedia page

In [0]:
data = requests.get("https://en.wikipedia.org/wiki/Tagaytay").text

In [0]:
soup = BeautifulSoup(data, 'html.parser')

In [0]:
brgyList = []

Getting the list of Baranggays in Tagaytay City

Baranggay is a term that often refers to an inner city neighbourhood, a suburb or a suburban neighborhood. -Wikipedia

In [0]:
for row in soup.find_all("div", class_="div-col columns column-width")[0].findAll("li"):
    brgyList.append(row.text)

In [0]:
df = pd.DataFrame({"Baranggay": brgyList})

Get the geographical coordinates

In [0]:
# define a function to get coordinates
def get_latlng(x):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Tagaytay City, Cavite'.format(x))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
coords = [ get_latlng(x) for x in df['Baranggay'].tolist()]

In [0]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [0]:
df.to_csv("df.csv", index=False)

### Create a map of Tagaytay with its Baranggays

In [37]:
address = 'Tagaytay City, Philippines'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tagaytay City, Philippines {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tagaytay City, Philippines 14.2999631, 120.9577667.


In [0]:
# create map of Tagaytay using latitude and longitude values
map_tagaytay = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Baranggay']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_tagaytay)  

In [39]:
map_tagaytay

### Use the Foursquare API to explore the baranggays

In [113]:
# define Foursquare Credentials and Version
CLIENT_ID = 'deleted' # your Foursquare ID
CLIENT_SECRET = 'deleted' # your Foursquare Secret
VERSION = '20200518' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: deleted
CLIENT_SECRET:deleted


Getting the venues from the Foursquare api

In [0]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Baranggay']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

Creating the new dataframe that includes the venues from the Foursquare API

In [0]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Baranggay', 'Latitude', 'Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

In [44]:
print(venues_df.shape)
venues_df.head()

(1608, 7)


,Baranggay,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Asisan,14.09274,120.91267,Bag of Beans,14.094945,120.915100,Coffee Shop
1,Asisan,14.09274,120.91267,Sonia's garden,14.090010,120.910687,Garden
2,Asisan,14.09274,120.91267,Mercury Drug,14.097833,120.919780,Pharmacy
3,Asisan,14.09274,120.91267,Breakfast at Antonio's,14.088246,120.898533,Breakfast Spot
4,Asisan,14.09274,120.91267,"Eway's Grill, Tagaytay",14.098311,120.921632,Filipino Restaurant


Number of Venues for each Baranggay

In [45]:
venues_df.groupby(['Baranggay']).count()

,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
Baranggay,,,,,,
Asisan,38,38,38,38,38,38
Bagong Tubig,27,27,27,27,27,27
Calabuso,43,43,43,43,43,43
Dapdap East,39,39,39,39,39,39
Dapdap West,15,15,15,15,15,15
Francisco,52,52,52,52,52,52
Guinhawa North,17,17,17,17,17,17
Guinhawa South,29,29,29,29,29,29
Iruhin East,13,13,13,13,13,13


In [97]:
venues_df.head(10)

,Baranggay,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Asisan,14.09274,120.91267,Bag of Beans,14.094945,120.915100,Coffee Shop
1,Asisan,14.09274,120.91267,Sonia's garden,14.090010,120.910687,Garden
2,Asisan,14.09274,120.91267,Mercury Drug,14.097833,120.919780,Pharmacy
3,Asisan,14.09274,120.91267,Breakfast at Antonio's,14.088246,120.898533,Breakfast Spot
4,Asisan,14.09274,120.91267,"Eway's Grill, Tagaytay",14.098311,120.921632,Filipino Restaurant
5,Asisan,14.09274,120.91267,Balinsasayaw Restaurant & Casitas,14.099800,120.922802,Filipino Restaurant
6,Asisan,14.09274,120.91267,"Bulaluhan, Tagaytay City",14.097647,120.918785,Diner
7,Asisan,14.09274,120.91267,Café Carmencita,14.099285,120.927749,Café
8,Asisan,14.09274,120.91267,Hotel Monticello,14.099965,120.928322,Hotel
9,Asisan,14.09274,120.91267,Nanay Dorie's Bulalohan,14.099850,120.924033,Filipino Restaurant


### Merge all types of restaurants into 1 venue category only

In [0]:
conds = ['Filipino Restaurant',
 'Restaurant',
 'Asian Restaurant',
 'Fast Food Restaurant',
 'Southern / Soul Food Restaurant',
 'Greek Restaurant',
 'American Restaurant',
 'Thai Restaurant',
 'Korean Restaurant',
 'Spanish Restaurant',
 'Japanese Restaurant',
 'Italian Restaurant',
 'Vietnamese Restaurant',
 'German Restaurant',
 'Seafood Restaurant',
 'Chinese Restaurant',
 'Mexican Restaurant']
clean_df = venues_df.copy()
clean_df.loc[clean_df['Venue Category'].isin(conds), 'Venue Category'] = 'Restaurant'

In [98]:
clean_df.head(10)

,Baranggay,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Asisan,14.09274,120.91267,Bag of Beans,14.094945,120.915100,Coffee Shop
1,Asisan,14.09274,120.91267,Sonia's garden,14.090010,120.910687,Garden
2,Asisan,14.09274,120.91267,Mercury Drug,14.097833,120.919780,Pharmacy
3,Asisan,14.09274,120.91267,Breakfast at Antonio's,14.088246,120.898533,Breakfast Spot
4,Asisan,14.09274,120.91267,"Eway's Grill, Tagaytay",14.098311,120.921632,Restaurant
5,Asisan,14.09274,120.91267,Balinsasayaw Restaurant & Casitas,14.099800,120.922802,Restaurant
6,Asisan,14.09274,120.91267,"Bulaluhan, Tagaytay City",14.097647,120.918785,Diner
7,Asisan,14.09274,120.91267,Café Carmencita,14.099285,120.927749,Café
8,Asisan,14.09274,120.91267,Hotel Monticello,14.099965,120.928322,Hotel
9,Asisan,14.09274,120.91267,Nanay Dorie's Bulalohan,14.099850,120.924033,Restaurant


### Analyze each Baranggay

In [100]:
# one hot encoding
df_onehot = pd.get_dummies(clean_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Baranggay'] = clean_df['Baranggay'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

print(df_onehot.shape)
df_onehot.head()

(1608, 87)


,Baranggay,ATM,Art Gallery,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Business Service,Cafeteria,Café,Campground,Casino,Cheese Shop,Coffee Shop,Convenience Store,Convention Center,Deli / Bodega,Dessert Shop,Diner,Dog Run,Donut Shop,Drugstore,Electronics Store,Event Space,Farm,Farmers Market,Flea Market,Fried Chicken Joint,Fruit & Vegetable Store,Garden,Golf Course,...,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indoor Play Area,Lake,Lounge,Market,Mini Golf,Mobile Phone Shop,Motel,Museum,Nature Preserve,Neighborhood,Other Great Outdoors,Park,Pastry Shop,Pharmacy,Pie Shop,Pizza Place,Plaza,Pool,Pub,Residential Building (Apartment / Condo),Resort,Restaurant,Scenic Lookout,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Theme Park,Theme Park Ride / Attraction,Trail,Train Station,Wine Shop,Wings Joint
0,Asisan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Asisan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Asisan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Asisan,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Asisan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [101]:
df_groupded = df_onehot.groupby(["Baranggay"]).mean().reset_index()

print(df_groupded.shape)
df_groupded.head()

(34, 87)


,Baranggay,ATM,Art Gallery,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Business Service,Cafeteria,Café,Campground,Casino,Cheese Shop,Coffee Shop,Convenience Store,Convention Center,Deli / Bodega,Dessert Shop,Diner,Dog Run,Donut Shop,Drugstore,Electronics Store,Event Space,Farm,Farmers Market,Flea Market,Fried Chicken Joint,Fruit & Vegetable Store,Garden,Golf Course,...,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indoor Play Area,Lake,Lounge,Market,Mini Golf,Mobile Phone Shop,Motel,Museum,Nature Preserve,Neighborhood,Other Great Outdoors,Park,Pastry Shop,Pharmacy,Pie Shop,Pizza Place,Plaza,Pool,Pub,Residential Building (Apartment / Condo),Resort,Restaurant,Scenic Lookout,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Theme Park,Theme Park Ride / Attraction,Trail,Train Station,Wine Shop,Wings Joint
0,Asisan,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.026316,0.000000,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0.026316,0.000000,0.0,0.0,0.026316,0.052632,0.0,0.0,0.0,0.026316,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.052632,0.000000,...,0.026316,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.026316,0.0,0.0,0.0,0.0,0.0,0.026316,0.0,0.0,0.000000,0.0,0.0,0.000000,0.052632,0.526316,0.000000,0.0,0.0,0.0,0.026316,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.026316
1,Bagong Tubig,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.111111,0.000000,0.037037,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.111111,0.000000,...,0.074074,0.000000,0.0,0.0,0.000000,0.037037,0.037037,0.0,0.000000,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.037037,0.481481,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.037037,0.000000,0.0,0.000000
2,Calabuso,0.0,0.0,0.023256,0.0,0.0,0.023256,0.0,0.0,0.046512,0.023256,0.023256,0.0,0.0,0.0,0.0,0.023256,0.0,0.023256,0.023256,0.0,0.0,0.023256,0.023256,0.0,0.0,0.0,0.000000,0.023256,0.0,0.0,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.000000,0.093023,...,0.069767,0.023256,0.0,0.0,0.000000,0.023256,0.023256,0.0,0.023256,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.023256,0.0,0.0,0.023256,0.093023,0.162791,0.046512,0.0,0.0,0.0,0.000000,0.0,0.023256,0.0,0.0,0.0,0.000000,0.023256,0.0,0.000000
3,Dapdap East,0.0,0.0,0.025641,0.0,0.0,0.025641,0.0,0.0,0.051282,0.025641,0.025641,0.0,0.0,0.0,0.0,0.025641,0.0,0.025641,0.025641,0.0,0.0,0.025641,0.025641,0.0,0.0,0.0,0.000000,0.025641,0.0,0.0,0.0,0.0,0.025641,0.0,0.0,0.0,0.0,0.000000,0.051282,...,0.076923,0.025641,0.0,0.0,0.000000,0.025641,0.025641,0.0,0.025641,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.025641,0.0,0.0,0.000000,0.102564,0.179487,0.051282,0.0,0.0,0.0,0.000000,0.0,0.025641,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
4,Dapdap West,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.066667,0.000000,0.066667,0.0,0.0,0.0,0.0,0.066667,0.0,0.066667,0.000000,0.0,0.0,0.066667,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.133333,0.000000,0.0,0.0,0.066667,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0,0.0,0.000000,0.200000,0.066667,0.066667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000


In [0]:
df_res_only = df_groupded[['Baranggay','Restaurant']]

In [103]:
df_res_only.head()

,Baranggay,Restaurant
0,Asisan,0.526316
1,Bagong Tubig,0.481481
2,Calabuso,0.162791
3,Dapdap East,0.179487
4,Dapdap West,0.066667


### Cluster Neighborhoods

Run k-means to cluster the Baranggay into 3 Clusters

In [104]:
# set number of clusters
kclusters = 3

kl_clustering = df_res_only.drop(["Baranggay"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 0, 2, 0, 0, 1, 2, 0], dtype=int32)

In [0]:
df_merged = df_res_only.copy()
df_merged["Cluster Labels"] = kmeans.labels_

In [106]:
df_merged = df_merged.join(df.set_index("Baranggay"), on="Baranggay")
print(df_merged.shape)
df_merged.head()

(34, 5)


,Baranggay,Restaurant,Cluster Labels,Latitude,Longitude
0,Asisan,0.526316,1,14.092740,120.912670
1,Bagong Tubig,0.481481,1,14.084370,120.897550
2,Calabuso,0.162791,0,14.141598,121.024709
3,Dapdap East,0.179487,0,14.146680,121.022710
4,Dapdap West,0.066667,2,14.148700,121.014560


In [107]:
df_merged.sort_values(["Cluster Labels"], inplace=True)
df_merged

,Baranggay,Restaurant,Cluster Labels,Latitude,Longitude
10,Iruhin West,0.151515,0,14.147060,120.996970
26,San Jose,0.162162,0,14.126030,120.970900
14,Mag-Asawang Ilat,0.181818,0,14.131270,120.968730
13,Kaybagal South (Poblacion),0.166667,0,14.116630,120.937110
18,Maitim 2nd East,0.235294,0,14.122790,120.961050
11,Kaybagal Central,0.193548,0,14.116200,120.937220
19,Maitim 2nd West,0.211268,0,14.125570,120.958850
9,Iruhin South,0.166667,0,14.133570,121.007950
28,Silang Junction South,0.277778,0,14.115070,120.961780
29,Sungay East,0.140000,0,14.126995,120.992159


### Visualize the resulting Clusters

In [108]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Baranggay'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
map_clusters.save('map_clusters.html')

### Examine Clusters

In [110]:
df_merged.loc[df_merged['Cluster Labels'] == 0]

,Baranggay,Restaurant,Cluster Labels,Latitude,Longitude
10,Iruhin West,0.151515,0,14.147060,120.996970
26,San Jose,0.162162,0,14.126030,120.970900
14,Mag-Asawang Ilat,0.181818,0,14.131270,120.968730
13,Kaybagal South (Poblacion),0.166667,0,14.116630,120.937110
18,Maitim 2nd East,0.235294,0,14.122790,120.961050
11,Kaybagal Central,0.193548,0,14.116200,120.937220
19,Maitim 2nd West,0.211268,0,14.125570,120.958850
9,Iruhin South,0.166667,0,14.133570,121.007950
28,Silang Junction South,0.277778,0,14.115070,120.961780
29,Sungay East,0.140000,0,14.126995,120.992159


In [111]:
df_merged.loc[df_merged['Cluster Labels'] == 1]

,Baranggay,Restaurant,Cluster Labels,Latitude,Longitude
0,Asisan,0.526316,1,14.09274,120.91267
25,Sambong,0.468085,1,14.09372,120.91421
27,Silang Junction North,0.353535,1,14.10852,120.95587
24,Patutong Malaki South,0.447368,1,14.10925,120.91169
16,Maharlika West,0.440000,1,14.09687,120.93250
21,Mendez Crossing West,0.460000,1,14.10407,120.91486
20,Mendez Crossing East,0.450000,1,14.10325,120.93278
15,Maharlika East,0.380000,1,14.10222,120.94879
7,Guinhawa South,0.413793,1,14.10345,120.90377
1,Bagong Tubig,0.481481,1,14.08437,120.89755


In [112]:
df_merged.loc[df_merged['Cluster Labels'] == 2]

,Baranggay,Restaurant,Cluster Labels,Latitude,Longitude
23,Patutong Malaki North,0.000000,2,14.11522,120.91027
12,Kaybagal North,0.000000,2,14.12508,120.93446
8,Iruhin East,0.076923,2,14.13857,121.01436
4,Dapdap West,0.066667,2,14.14870,121.01456


### Observations

Based from the clusters formed, majority of restaurants are located on the central part of the city. It can be seen that majority of restaurant are in clusters 0 while a minimum number of restaurants can be seen at cluster 2. This means that there is a high potential to open a new restaurants on baranggays on cluster 2 as it has the minimum number of restaurant or potential competitions.